## Introduction

This project replicates the counterfactual part of Multidimensional Auctions of Contracts: An Empirical Analysis by Yunmi Kong, Isabelle Perrigne, and Quang Vuong published in American Economics Review in 2022. This paper analyzes cash-royalty auctions of Louisiana oil leases. In this auction, the bidders bid for a one-time cash payment and royalty (the proportion of revenues they will pay the government afterward). The winner of the auction has the right of drilling, but it is not compulsory. They model this auction as a private value auction in which each bidder draws a private productivity level $\theta_1$ and a private cost $\theta_2$ of drilling $ from a joint distribution. The private value also depends on the oil price, which is modeled as a geometric Brownian motion. I will include more details of their auction model in this notebook.

This paper estimates the joint distribution of $\theta_1$ and $\theta_2$ non-parametrically and analyzes counterfactuals based on that. However, in my replication, I skip the estimation part and do the counterfactual only. First, I simulate $\theta_1$ and $\theta_2$ and compute the private values and bids for each firm. Then, I simulate the expected revenues of the government under different scenarios as follows.
1. The auction has flexible royalty.
2. The auctions have fixed royalty.
3. The government has different rules for selecting the winner.
4. The oil price increases.


In [1]:
import numpy as np

